In [ ]:
!pip install imutils
import keras
import tensorflow
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout, Input, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
from keras.models import Model
from cv2 import imread
import numpy as np
import pandas as pd
print("Imported Modules Successfully")

In [ ]:
data = []
labels = []
width, height=150,150

imagePaths = list(paths.list_images('../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images'))
data = []
labels = []
#print(imagePaths)
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    #print(imagePath)
    image = load_img(imagePath, target_size=(width, height))
    image = img_to_array(image)
    data.append(image)
    labels.append(label)

data = np.array(data, dtype="float32")
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
#print(labels)
#labels = to_categorical(labels)

data, labels = shuffle(data, labels)

print(data.shape)
print(labels.shape)

In [ ]:
test_ratio = 0.2

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = test_ratio)

print("Train images:",x_train.shape)
print("Test images:",x_test.shape)
print("Train label:",y_train.shape)
print("Test label:",y_test.shape)

In [ ]:
from keras.models import Model
from keras.layers import Input 
from keras.applications.vgg16 import VGG16
from keras import layers

INIT_LR = 1e-3
EPOCHS = 40
BS = 32

opt = Adam(learning_rate = 1e-3)
base_model = VGG16(input_shape = (150, 150, 3),include_top = False,weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = Sequential([base_model,
                                 keras.layers.GlobalAveragePooling2D(),
                                 keras.layers.Dropout(0.2),
                                 keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

In [ ]:
history = model.fit(x_train, y_train, epochs = EPOCHS, validation_data = (x_test, y_test), callbacks=[EarlyStopping, model_save,rlr])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(12)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(x_test)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
y_test1 = [list(x).index(1) for x in y_test]
print(classification_report(y_test1, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")

In [ ]:
model = Sequential()
#conv=> relu => max pool
model.add(Conv2D(32, (3, 3), padding = 'same', activation = 'relu', input_shape = (150, 150, 3)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation = 'softmax'))
#return fully constructed model 
opt = Adam(learning_rate = 1e-3)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model2.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

In [ ]:
history = model.fit(x_train, y_train, epochs = EPOCHS, validation_data = (x_test, y_test), callbacks = [EarlyStopping, rlr, model_save])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(13)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(x_test)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
y_test1 = [list(x).index(1) for x in y_test]
print(classification_report(y_test1, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")

In [ ]:
#Transfer Learning Part
from tensorflow.keras.applications import MobileNetV2
base_model = MobileNetV2(input_shape = (150, 150, 3),include_top = False,weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = Sequential([base_model,
                                 keras.layers.GlobalAveragePooling2D(),
                                 keras.layers.Dropout(0.2),
                                 keras.layers.Dense(5, activation="softmax")                                     
                                ])

model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

rlr = ReduceLROnPlateau( monitor="val_accuracy",
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode="max",
                            min_delta=0.01)

model_save = ModelCheckpoint('./Model3.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

In [ ]:
history = model.fit(x_train, y_train, epochs = EPOCHS, validation_data = (x_test, y_test), callbacks = [EarlyStopping, rlr, model_save])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(19)

plt.figure(figsize = (15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(x_test)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]
y_test1 = [list(x).index(1) for x in y_test]
print(classification_report(y_test1, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")

In [ ]:
from tensorflow.keras.applications import Xception
base_model = Xception(input_shape = (150, 150, 3),include_top = False,weights = "imagenet")
#prevent weight update while training
for layer in base_model.layers:
    layer.trainable =  False
#add our layers
model = Sequential([base_model,keras.layers.GlobalMaxPooling2D(),keras.layers.Dropout(0.2),keras.layers.Dense(5, activation="softmax")                                     ])
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
model.summary()